Final Notebook used to create monthly metrics for all of the monthly twitter data 

Required: 
1.Google Drive to be able to include the dataset 


In [5]:
# -*- coding: utf-8 -*-

#Installing textblob is the text sentiment model
#corpora is the models pretrained set
!pip install textblob
!python -m textblob.download_corpora

#Dask is an extension of pandas thats good at reading big data csvs
!pip install dask
!python -m pip install dask[dataframe]
!pip install google

#Sentiment analysis model
from textblob import TextBlob

#DataFrame support
from dask import dataframe as dd
from pandas.errors import EmptyDataError
import pandas as pd
import numpy as np

#Writing csvs
import csv
#Dealing with Operation System
import os
#Dealing with input output
import io
#Regular expressions
import re
#Timing module
import time
#Library for parsing dictionaries from strings
import ast
import json


rawDataFolder = "./rawTweetData/"
tweetSumsFolder = "./tweet_sums/"
noRTFolder = "./TweetsCleanedNoRetweets/"
cleanEscapeFolder = "./cleanEscapeChars/"




[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


To get the dataset
Make sure that you get the data from our google drive and add it to your google drive.

This is so you can mount the folder into here

Be sure to copy the whole structure

Inside should consist of 2 directories
./CleanedNoRetweets/
./tweet_sums/

Also includes 256 csv files and 1 textfile holding all files text names 

https://drive.google.com/drive/folders/1lX3chADHG6LRpkgFgsp1OAi3Gyh4-z_q?usp=sharing

In [1]:
!hostname

5d79407b0ece


In [5]:
#Need to get the data csvs from here

from google.colab import drive
from pathlib import Path
import shutil


#Change this when you have your data in your drive 
drive.mount('/content/drive')

#Rename the directory link with your directory
directoryLink = '4906DataScienceCleanedCSVs/'
os.chdir("/content/drive/MyDrive/"+directoryLink)
os.chdir("../../../")
print(os.listdir())

print(os.path.isdir('TweetsCleanedNoRetweets'))

if not os.path.isdir('TweetsCleanedNoRetweets'):
   os.mkdir('TweetsCleanedNoRetweets')

if not os.path.isdir('tweet_sums'):
  os.mkdir('tweet_sums')
else:
  shutil.rmtree("tweet_sums")
  os.mkdir('tweet_sums')


if not os.path.isdir('rawTweetData'):
  os.mkdir('rawTweetData')


if not os.path.isdir('cleanEscapeChars'):
  os.mkdir('cleanEscapeChars')
else:
  shutil.rmtree("cleanEscapeChars")
  os.mkdir('cleanEscapeChars')


source_folder = r"./drive/MyDrive/"+directoryLink
destination_folder = r"./rawTweetData/"

start = time.time()
# fetch all files
for file_name in os.listdir(source_folder):
    
    # construct full file path
    source = source_folder + file_name
    destination = destination_folder + file_name
    # copy only files
    if os.path.isfile(source):
        shutil.copy(source, destination)
        print('copied', file_name)


end= time.time()

print("Time to copy files over :" + str((end-start)) +" seconds")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['.config', 'drive', 'tweet_sums', 'cleanEscapeChars', 'TweetsCleanedNoRetweets', 'rawTweetData', 'sample_data']
True
copied The Sandbox_3_2021Tweets.csv
copied Parallel Alpha_11_2021Tweets.csv
copied World of Women_6_2021Tweets.csv
copied Bored Ape Kennel Club_10_2021Tweets.csv
copied Bored Ape Kennel Club_5_2021Tweets.csv
copied Sorare_9_2021Tweets.csv
copied Sorare_8_2021Tweets.csv
copied The Sandbox_5_2021Tweets.csv
copied Parallel Alpha_8_2021Tweets.csv
copied Sorare_11_2021Tweets.csv
copied The Sandbox_2_2021Tweets.csv
copied The Sandbox_7_2021Tweets.csv
copied Sorare_2_2021Tweets.csv
copied Bored Ape Kennel Club_4_2021Tweets.csv
copied Bored Ape Kennel Club_3_2021Tweets.csv
copied Bored Ape Kennel Club_7_2021Tweets.csv
copied Parallel Alpha_5_2021Tweets.csv
copied World of Women_5_2021Tweets.csv
copied crypToadz_6_2021Tweets.csv
copied MutantApeYachtCl

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
##Clean the tweet of any links, pictures or extra emojis
def clean_tweet(tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split())


##Function to determine the tweets sentiment
def get_tweet_sentiment_plus_count(row):
        '''
    Returns the sum of two decimal numbers in binary digits.

            Parameters:
                    tweet (DataFrame row): pandas row object 

            Returns:
                    result (str): String with the tweets sentiment
    '''
        reformatMetrics = row['public_metrics'].replace('"','')
        metric = ast.literal_eval(reformatMetrics)
        media = 0
        if type(row['attachments']) is str and not row['attachments']:
                media = 1        
        
        analysis = TextBlob(clean_tweet(row['text']))
        sentiment=""
        # set sentiment
        if analysis.sentiment.polarity > 0:
                sentiment =  'positive'
        elif analysis.sentiment.polarity == 0:
                sentiment =  'neutral'
        else:
                sentiment = 'negative'
                
        return [metric['retweet_count'],metric['reply_count'],metric['like_count'],metric['quote_count'], sentiment,media]        



In [4]:

#All the file titles to be cleaned and processed
fileTitles=["artblocks_10_2021Tweets.csv",
"artblocks_11_2021Tweets.csv",
"artblocks_12_2021Tweets.csv",
"artblocks_1_2021Tweets.csv",
"artblocks_1_2022Tweets.csv",
"artblocks_2_2021Tweets.csv",
"artblocks_2_2022Tweets.csv",
"artblocks_3_2021Tweets.csv",
"artblocks_4_2021Tweets.csv",
"artblocks_5_2021Tweets.csv",
"artblocks_6_2021Tweets.csv",
"artblocks_7_2021Tweets.csv",
"artblocks_8_2021Tweets.csv",
"artblocks_9_2021Tweets.csv",
"AxieInfinity_10_2021Tweets.csv",
"AxieInfinity_11_2021Tweets.csv",
"AxieInfinity_12_2021Tweets.csv",
"AxieInfinity_1_2021Tweets.csv",
"AxieInfinity_1_2022Tweets.csv",
"AxieInfinity_2_2021Tweets.csv",
"AxieInfinity_2_2022Tweets.csv",
"AxieInfinity_3_2021Tweets.csv",
"AxieInfinity_4_2021Tweets.csv",
"AxieInfinity_5_2021Tweets.csv",
"AxieInfinity_6_2021Tweets.csv",
"AxieInfinity_7_2021Tweets.csv",
"AxieInfinity_8_2021Tweets.csv",
"AxieInfinity_9_2021Tweets.csv",
"Azuki_10_2021Tweets.csv",
"Azuki_11_2021Tweets.csv",
"Azuki_12_2021Tweets.csv",
"Azuki_1_2022Tweets.csv",
"Azuki_2_2022Tweets.csv",
"Bored Ape Kennel Club_10_2021Tweets.csv",
"Bored Ape Kennel Club_11_2021Tweets.csv",
"Bored Ape Kennel Club_12_2021Tweets.csv",
"Bored Ape Kennel Club_1_2022Tweets.csv",
"Bored Ape Kennel Club_2_2022Tweets.csv",
"Bored Ape Kennel Club_4_2021Tweets.csv",
"Bored Ape Kennel Club_5_2021Tweets.csv",
"Bored Ape Kennel Club_6_2021Tweets.csv",
"Bored Ape Kennel Club_7_2021Tweets.csv",
"Bored Ape Kennel Club_8_2021Tweets.csv",
"Bored Ape Kennel Club_9_2021Tweets.csv",
"BoredApeYachtClub_10_2021Tweets.csv",
"BoredApeYachtClub_11_2021Tweets.csv",
"BoredApeYachtClub_12_2021Tweets.csv",
"BoredApeYachtClub_1_2022Tweets.csv",
"BoredApeYachtClub_2_2022Tweets.csv",
"BoredApeYachtClub_5_2021Tweets.csv",
"BoredApeYachtClub_6_2021Tweets.csv",
"BoredApeYachtClub_7_2021Tweets.csv",
"BoredApeYachtClub_8_2021Tweets.csv",
"BoredApeYachtClub_9_2021Tweets.csv",
"CloneX_10_2021Tweets.csv",
"CloneX_11_2021Tweets.csv",
"CloneX_12_2021Tweets.csv",
"CloneX_1_2022Tweets.csv",
"CloneX_2_2022Tweets.csv",
"CloneX_9_2021Tweets.csv",
"Cool Cats_10_2021Tweets.csv",
"Cool Cats_11_2021Tweets.csv",
"Cool Cats_12_2021Tweets.csv",
"Cool Cats_1_2022Tweets.csv",
"Cool Cats_2_2022Tweets.csv",
"Cool Cats_4_2021Tweets.csv",
"Cool Cats_5_2021Tweets.csv",
"Cool Cats_6_2021Tweets.csv",
"Cool Cats_7_2021Tweets.csv",
"Cool Cats_8_2021Tweets.csv",
"Cool Cats_9_2021Tweets.csv",
"crypToadz_10_2021Tweets.csv",
"crypToadz_11_2021Tweets.csv",
"crypToadz_12_2021Tweets.csv",
"crypToadz_1_2022Tweets.csv",
"crypToadz_2_2022Tweets.csv",
"crypToadz_6_2021Tweets.csv",
"crypToadz_7_2021Tweets.csv",
"crypToadz_8_2021Tweets.csv",
"crypToadz_9_2021Tweets.csv",
"Cryptopunk_10_2021Tweets.csv",
"Cryptopunk_11_2021Tweets.csv",
"Cryptopunk_12_2021Tweets.csv",
"Cryptopunk_1_2021Tweets.csv",
"Cryptopunk_1_2022Tweets.csv",
"Cryptopunk_2_2021Tweets.csv",
"Cryptopunk_2_2022Tweets.csv",
"Cryptopunk_3_2021Tweets.csv",
"Cryptopunk_4_2021Tweets.csv",
"Cryptopunk_5_2021Tweets.csv",
"Cryptopunk_6_2021Tweets.csv",
"Cryptopunk_7_2021Tweets.csv",
"Cryptopunk_8_2021Tweets.csv",
"Cryptopunk_9_2021Tweets.csv",
"CyberKongs_10_2021Tweets.csv",
"CyberKongs_11_2021Tweets.csv",
"CyberKongs_12_2021Tweets.csv",
"CyberKongs_1_2021Tweets.csv",
"CyberKongs_1_2022Tweets.csv",
"CyberKongs_2_2021Tweets.csv",
"CyberKongs_2_2022Tweets.csv",
"CyberKongs_3_2021Tweets.csv",
"CyberKongs_4_2021Tweets.csv",
"CyberKongs_5_2021Tweets.csv",
"CyberKongs_6_2021Tweets.csv",
"CyberKongs_7_2021Tweets.csv",
"CyberKongs_8_2021Tweets.csv",
"CyberKongs_9_2021Tweets.csv",
"Doodles_10_2021Tweets.csv",
"Doodles_11_2021Tweets.csv",
"Doodles_12_2021Tweets.csv",
"Doodles_1_2022Tweets.csv",
"Doodles_2_2022Tweets.csv",
"Doodles_7_2021Tweets.csv",
"Doodles_8_2021Tweets.csv",
"Doodles_9_2021Tweets.csv",
"Famers World_10_2021Tweets.csv",
"Famers World_11_2021Tweets.csv",
"Famers World_12_2021Tweets.csv",
"Famers World_1_2022Tweets.csv",
"Famers World_2_2022Tweets.csv",
"Famers World_4_2021Tweets.csv",
"Famers World_5_2021Tweets.csv",
"Famers World_6_2021Tweets.csv",
"Famers World_7_2021Tweets.csv",
"Famers World_8_2021Tweets.csv",
"Famers World_9_2021Tweets.csv",
"Loot_10_2021Tweets.csv",
"Loot_11_2021Tweets.csv",
"Loot_12_2021Tweets.csv",
"Loot_1_2022Tweets.csv",
"Loot_2_2022Tweets.csv",
"Loot_5_2021Tweets.csv",
"Loot_6_2021Tweets.csv",
"Loot_7_2021Tweets.csv",
"Loot_8_2021Tweets.csv",
"Loot_9_2021Tweets.csv",
"Meebits_10_2021Tweets.csv",
"Meebits_11_2021Tweets.csv",
"Meebits_12_2021Tweets.csv",
"Meebits_1_2022Tweets.csv",
"Meebits_2_2021Tweets.csv",
"Meebits_2_2022Tweets.csv",
"Meebits_3_2021Tweets.csv",
"Meebits_4_2021Tweets.csv",
"Meebits_5_2021Tweets.csv",
"Meebits_6_2021Tweets.csv",
"Meebits_7_2021Tweets.csv",
"Meebits_8_2021Tweets.csv",
"Meebits_9_2021Tweets.csv",
"MutantApeYachtClub_10_2021Tweets.csv",
"MutantApeYachtClub_11_2021Tweets.csv",
"MutantApeYachtClub_12_2021Tweets.csv",
"MutantApeYachtClub_1_2022Tweets.csv",
"MutantApeYachtClub_2_2022Tweets.csv",
"MutantApeYachtClub_5_2021Tweets.csv",
"MutantApeYachtClub_6_2021Tweets.csv",
"MutantApeYachtClub_7_2021Tweets.csv",
"MutantApeYachtClub_8_2021Tweets.csv",
"MutantApeYachtClub_9_2021Tweets.csv",
"Parallel Alpha_10_2021Tweets.csv",
"Parallel Alpha_11_2021Tweets.csv",
"Parallel Alpha_12_2021Tweets.csv",
"Parallel Alpha_1_2022Tweets.csv",
"Parallel Alpha_2_2022Tweets.csv",
"Parallel Alpha_3_2021Tweets.csv",
"Parallel Alpha_4_2021Tweets.csv",
"Parallel Alpha_5_2021Tweets.csv",
"Parallel Alpha_6_2021Tweets.csv",
"Parallel Alpha_7_2021Tweets.csv",
"Parallel Alpha_8_2021Tweets.csv",
"Parallel Alpha_9_2021Tweets.csv",
"Sorare_10_2021Tweets.csv",
"Sorare_11_2021Tweets.csv",
"Sorare_12_2021Tweets.csv",
"Sorare_1_2021Tweets.csv",
"Sorare_1_2022Tweets.csv",
"Sorare_2_2021Tweets.csv",
"Sorare_2_2022Tweets.csv",
"Sorare_3_2021Tweets.csv",
"Sorare_4_2021Tweets.csv",
"Sorare_5_2020Tweets.csv",
"Sorare_5_2021Tweets.csv",
"Sorare_6_2021Tweets.csv",
"Sorare_7_2021Tweets.csv",
"Sorare_8_2021Tweets.csv",
"Sorare_9_2021Tweets.csv",
"The Sandbox_10_2021Tweets.csv",
"The Sandbox_11_2021Tweets.csv",
"The Sandbox_12_2021Tweets.csv",
"The Sandbox_1_2021Tweets.csv",
"The Sandbox_1_2022Tweets.csv",
"The Sandbox_2_2021Tweets.csv",
"The Sandbox_2_2022Tweets.csv",
"The Sandbox_3_2021Tweets.csv",
"The Sandbox_4_2021Tweets.csv",
"The Sandbox_5_2021Tweets.csv",
"The Sandbox_6_2021Tweets.csv",
"The Sandbox_7_2021Tweets.csv",
"The Sandbox_8_2021Tweets.csv",
"The Sandbox_9_2021Tweets.csv",
"VeeFriends_10_2021Tweets.csv",
"VeeFriends_11_2021Tweets.csv",
"VeeFriends_12_2021Tweets.csv",
"VeeFriends_1_2022Tweets.csv",
"VeeFriends_2_2021Tweets.csv",
"VeeFriends_2_2022Tweets.csv",
"VeeFriends_3_2021Tweets.csv",
"VeeFriends_4_2021Tweets.csv",
"VeeFriends_5_2021Tweets.csv",
"VeeFriends_6_2021Tweets.csv",
"VeeFriends_7_2021Tweets.csv",
"VeeFriends_8_2021Tweets.csv",
"VeeFriends_9_2021Tweets.csv",
"World of Women_10_2021Tweets.csv",
"World of Women_11_2021Tweets.csv",
"World of Women_12_2021Tweets.csv",
"World of Women_1_2022Tweets.csv",
"World of Women_2_2022Tweets.csv",
"World of Women_4_2021Tweets.csv",
"World of Women_5_2021Tweets.csv",
"World of Women_6_2021Tweets.csv",
"World of Women_7_2021Tweets.csv",
"World of Women_8_2021Tweets.csv",
"World of Women_9_2021Tweets.csv"]

In [8]:
import json
from pandas.errors import EmptyDataError
##Find unique Tweets that are not retweets
## IE retweets start with RT


print("Cleaning unescaped quotes")
start = time.time()
for unCleanFile in fileTitles:
  with open(rawDataFolder+unCleanFile, 'r', encoding='utf-8', newline='') as file1, open(cleanEscapeFolder+unCleanFile[:-4]+"Escaped.csv",'w', encoding='utf-8', newline='') as file2:
    reader = csv.reader(file1)
     # Write column headers without quotes
    headers = next(reader)
    str1 = ''.join(headers)
    writer = csv.writer(file2)
    writer.writerow(headers)

    # Write all other rows with quotes
    writer = csv.writer(file2, quoting=csv.QUOTE_ALL,delimiter=',')
    for row in reader:
        if len(row) > 3:
          row[4] = clean_tweet(row[4])
          row[4] = row[4].replace('"','')
          row[0] = json.dumps(row[0])
          row[3] = json.dumps(row[3])
        writer.writerow(row)

end= time.time()
print("Done cleaning in "+ str(end-start) + " seconds")


Cleaning unescaped quotes
Done cleaning in 227.8302264213562 seconds


In [6]:

#### result containers
good_files = []
empty_files = []


print("Start parsing out empty files")
start = time.time()
#### iterate files in directo
for one_file in fileTitles:
        print(f"Parsing: {one_file}")
        try:
        #### read csv file to pandas data frame
                df = dd.read_csv(cleanEscapeFolder+one_file[:-4]+"Escaped.csv", low_memory=False, encoding='utf-8', header=0, quoting=1)
                good_files.append(one_file)
        except EmptyDataError:
                print(f"No columns to parse from file {one_file}")
                empty_files.append(one_file)
end = time.time()
print('Done parsing in '+ str(end-start) +" seconds")


Start parsing out empty files
Parsing: artblocks_10_2021Tweets.csv
Parsing: artblocks_11_2021Tweets.csv
Parsing: artblocks_12_2021Tweets.csv
Parsing: artblocks_1_2021Tweets.csv
Parsing: artblocks_1_2022Tweets.csv
Parsing: artblocks_2_2021Tweets.csv
Parsing: artblocks_2_2022Tweets.csv
Parsing: artblocks_3_2021Tweets.csv
Parsing: artblocks_4_2021Tweets.csv
Parsing: artblocks_5_2021Tweets.csv
Parsing: artblocks_6_2021Tweets.csv
Parsing: artblocks_7_2021Tweets.csv
Parsing: artblocks_8_2021Tweets.csv
Parsing: artblocks_9_2021Tweets.csv
Parsing: AxieInfinity_10_2021Tweets.csv
Parsing: AxieInfinity_11_2021Tweets.csv
Parsing: AxieInfinity_12_2021Tweets.csv
Parsing: AxieInfinity_1_2021Tweets.csv
Parsing: AxieInfinity_1_2022Tweets.csv
Parsing: AxieInfinity_2_2021Tweets.csv
Parsing: AxieInfinity_2_2022Tweets.csv
Parsing: AxieInfinity_3_2021Tweets.csv
Parsing: AxieInfinity_4_2021Tweets.csv
Parsing: AxieInfinity_5_2021Tweets.csv
Parsing: AxieInfinity_6_2021Tweets.csv
Parsing: AxieInfinity_7_2021Twe

In [10]:

##Get the columns for the new Dataframe
df = dd.read_csv(cleanEscapeFolder+'BoredApeYachtClub_6_2021TweetsEscaped.csv', quotechar='"')
errorFiles = []

columnCheck =[]

print("Starting to strip ReTweets")
start = time.time()
###This is produce a csv of no RT data
for i in good_files:
        dfNewTitle = i[:-4] + "NoRT.csv"
        if i in good_files:
          #Start timer
          start = time.time()
          #Get the dataframe as partitions
          dfVal = dd.read_csv(cleanEscapeFolder+i[:-4]+"Escaped.csv", dtype={'attachments': object},)
          uniqueTweetsStrings = []
          #Return subset of dataframe with rows where first 2 letters of text != RT
          df2 = dfVal[(dfVal.text.str[:2] != "RT")]
          end = time.time()
          #Used for debugging
          columnCheck.append([df2.columns, dfNewTitle])
          print(dfNewTitle + " Time elapsed: " + str((end - start)))
          print(dfNewTitle)
          df2.to_csv(noRTFolder+dfNewTitle, single_file = True,encoding='utf-8')                        

end = time.time()
print("Done Stripping ReTweets "+ str(end-start) +" seconds")


Starting to strip ReTweets
artblocks_10_2021TweetsNoRT.csv Time elapsed: 0.013129472732543945
artblocks_10_2021TweetsNoRT.csv
artblocks_11_2021TweetsNoRT.csv Time elapsed: 0.009130477905273438
artblocks_11_2021TweetsNoRT.csv
artblocks_12_2021TweetsNoRT.csv Time elapsed: 0.009207725524902344
artblocks_12_2021TweetsNoRT.csv
artblocks_1_2021TweetsNoRT.csv Time elapsed: 0.005159139633178711
artblocks_1_2021TweetsNoRT.csv
artblocks_1_2022TweetsNoRT.csv Time elapsed: 0.008697986602783203
artblocks_1_2022TweetsNoRT.csv
artblocks_2_2021TweetsNoRT.csv Time elapsed: 0.005444765090942383
artblocks_2_2021TweetsNoRT.csv
artblocks_2_2022TweetsNoRT.csv Time elapsed: 0.008968353271484375
artblocks_2_2022TweetsNoRT.csv
artblocks_3_2021TweetsNoRT.csv Time elapsed: 0.007314920425415039
artblocks_3_2021TweetsNoRT.csv
artblocks_4_2021TweetsNoRT.csv Time elapsed: 0.011069297790527344
artblocks_4_2021TweetsNoRT.csv
artblocks_5_2021TweetsNoRT.csv Time elapsed: 0.009534358978271484
artblocks_5_2021TweetsNoRT.c

In [15]:

print(fileTitles)
##Takes each no RT csv and aggreagtes some data
collectionData=[]
failedFiles =[]



collectionPrev = ''.join(filter(str.isalpha, fileTitles[0])) 

start = time.time()
counter = 1
for i in fileTitles:
  dfTotalNumTweets = 0
  noRTName= i[:-4] + "NoRT.csv"
  collection = ''.join(filter(str.isalpha, i))
  date = ''.join(filter(str.isdigit, i))
  print(date)
  print(collection,collectionPrev)
  #If we move onto a new collection, print the data that was generated into a csv
  if(collection != collectionPrev):
      
          print("Printint to csv "+tweetSumsFolder+collectionPrev[:-4]+"DataByMonth.csv")
          end = time.time()
          print("Time since start and done"+str(counter)+" Collections :"+str((end-start)))
          keys = collectionData[0].keys()
          ##Sort by month
          sorted(collectionData, key = lambda i: i['month'])
          with open(tweetSumsFolder+collectionPrev[:-3]+"DataByMonth.csv", 'w', newline='') as output_file:
                  dict_writer = csv.DictWriter(output_file, keys)
                  dict_writer.writeheader()
                  dict_writer.writerows(collectionData)
                  print(collectionPrev +"printing to csv")       
                  collectionPrev = collection
                  ##Clear last collections data for next collection
                  collectionData = []
          totalOriginalTweets = 0
        #Good files are files with rows
  if i in good_files:
          #Get the total tweets from the original data
          dfTotal = dd.read_csv(cleanEscapeFolder+i[:-4]+"Escaped.csv", encoding='utf-8', header=0, quoting=1)
          dfTotalNumTweets = dfTotal.shape[0].compute()

          #Take the cleaned data
          df = dd.read_csv(noRTFolder+noRTName, encoding='utf-8', header=0, quoting=1)
          sumTweets = 0
          pos = 0
          neut =0
          neg =0
          retweet = 0
          reply=0
          like=0
          quote=0
          media=0

          #Apply the function get_tweet_sentiment_plus_count to all rows
          result =  df.apply(get_tweet_sentiment_plus_count, axis=1, meta=(None, 'object'))
          ##Summation of results
          try:
            for val in result:
                try:
                  #Sum values
                  retweet+= val[0]
                  reply+= val[1]
                  like+= val[2]
                  quote+= val[3] 
                  media+= val[5]
                  sumTweets += 1
                  if(val[4] == "positive"):
                          pos += 1
                  elif (val[4] == "neutral"):
                          neut += 1
                  else:
                          neg += 1   
                except:
                  print("error on file "+noRTName)
                  failedFiles.append(noRTName)            
                  #Deal with sums and year
          except:
            print("Failed on file method "+noRTName)
            failedFiles.append(noRTName)  

          dateYear = date[-4:]
          dateMonth = date[:-4]
          date = dateMonth + "_"+ dateYear 

          #Add each months values into a dictionary and then into a list
          frame2 = {"month":date,"tweetsum":sumTweets, "positive":pos, "negative":neg, "neutral":neut, "retweet_count":retweet,"reply_count":reply,"like_count":like,"quote_count":quote,"media_links":media,"totalOriginalTweets":dfTotalNumTweets}
          collectionData.append(frame2.copy())                
  else:
          #If bad_files list, then print all zeros for data
          dateYear = date[-4:]
          dateMonth = date[:-4]
          date = dateMonth + "_"+ dateYear 
          frame2 = {"month":date,"tweetsum":0, "positive":0, "negative":0, "neutral":0, "retweet_count":0,"reply_count":0,"like_count":0,"quote_count":0,"media_links":0,"totalOriginalTweets":0}
          collectionData.append(frame2.copy())          


  #Deals with printing the last collection
  if(i == fileTitles[-1]):
          keys = collectionData[0].keys()
          with open(tweetSumsFolder+collection[:-4]+"DataByMonth.csv", 'w', newline='') as output_file:
                  dict_writer = csv.DictWriter(output_file, keys)
                  dict_writer.writeheader()
                  dict_writer.writerows(collectionData)
                  print(collectionPrev)
print("Done")
end=time.time()

print("Finished aggregation in : "+str(end-start)+" seconds")

for file in failedFiles:
  print("This file failed " + file)
                    


['artblocks_10_2021Tweets.csv', 'artblocks_11_2021Tweets.csv', 'artblocks_12_2021Tweets.csv', 'artblocks_1_2021Tweets.csv', 'artblocks_1_2022Tweets.csv', 'artblocks_2_2021Tweets.csv', 'artblocks_2_2022Tweets.csv', 'artblocks_3_2021Tweets.csv', 'artblocks_4_2021Tweets.csv', 'artblocks_5_2021Tweets.csv', 'artblocks_6_2021Tweets.csv', 'artblocks_7_2021Tweets.csv', 'artblocks_8_2021Tweets.csv', 'artblocks_9_2021Tweets.csv', 'AxieInfinity_10_2021Tweets.csv', 'AxieInfinity_11_2021Tweets.csv', 'AxieInfinity_12_2021Tweets.csv', 'AxieInfinity_1_2021Tweets.csv', 'AxieInfinity_1_2022Tweets.csv', 'AxieInfinity_2_2021Tweets.csv', 'AxieInfinity_2_2022Tweets.csv', 'AxieInfinity_3_2021Tweets.csv', 'AxieInfinity_4_2021Tweets.csv', 'AxieInfinity_5_2021Tweets.csv', 'AxieInfinity_6_2021Tweets.csv', 'AxieInfinity_7_2021Tweets.csv', 'AxieInfinity_8_2021Tweets.csv', 'AxieInfinity_9_2021Tweets.csv', 'Azuki_10_2021Tweets.csv', 'Azuki_11_2021Tweets.csv', 'Azuki_12_2021Tweets.csv', 'Azuki_1_2022Tweets.csv', 'Azu

In [8]:

print("FAILED Files")
for file in failedFiles:
        finalend = time.time()
print("The final time took "+str((finalend-start)))

FAILED Files
The final time took 1237.295583486557


In [13]:
!zip -r ./tweet_sums.zip ./tweet_sums
from google.colab import files
files.download("tweet_sums.zip")

  adding: tweet_sums/ (stored 0%)
  adding: tweet_sums/CloneXTweetsDataByMonth.csv (deflated 40%)
  adding: tweet_sums/LootTweetsDataByMonth.csv (deflated 46%)
  adding: tweet_sums/FamersWorldTweetsDataByMonth.csv (deflated 47%)
  adding: tweet_sums/VeeFriendsTweetsDataByMonth.csv (deflated 49%)
  adding: tweet_sums/MutantApeYachtClubTweetsDataByMonth.csv (deflated 45%)
  adding: tweet_sums/crypToadzTweetsDataByMonth.csv (deflated 47%)
  adding: tweet_sums/AxieInfinityTweetsDataByMonth.csv (deflated 47%)
  adding: tweet_sums/MeebitsTweetsDataByMonth.csv (deflated 72%)
  adding: tweet_sums/SorareTweetsDataByMonth.csv (deflated 48%)
  adding: tweet_sums/BoredApeYachtClubTweetsDataByMonth.csv (deflated 46%)
  adding: tweet_sums/WorldofWomenTweetDataByMonth.csv (deflated 57%)
  adding: tweet_sums/TheSandboxTweetsDataByMonth.csv (deflated 48%)
  adding: tweet_sums/DoodlesTweetsDataByMonth.csv (deflated 45%)
  adding: tweet_sums/CryptopunkTweetsDataByMonth.csv (deflated 51%)
  adding: tweet_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>